# Eigenmode simulation of single qubit with readout resonator

In [ ]:
%load_ext autoreload
%autoreload 2

## Rendering the design

In [ ]:
import names as n
import design as d
from qdesignoptimizer.utils.chip_generation import create_chip_base

design, gui = create_chip_base(n.CHIP_NAME, d.chip_type, open_gui=True)
d.render_qiskit_metal_design(design, gui)

## Creating the study and optimization targets

In [ ]:
import mini_studies as ms
import optimization_targets as ot

In [ ]:
MINI_STUDY_GROUP = n.NBR_1
MINI_STUDY = ms.get_mini_study_qb_res(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: d.render_qiskit_metal_design(design, gui)


opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[MINI_STUDY_GROUP],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=True,
    opt_target_resonator_qubit_chi=True,
    use_simple_resonator_qubit_chi_relation=True,
)

## Creating design analysis objects

In [ ]:
import time
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from design import CoupledLineTee_mesh_names
from qdesignoptimizer.design_analysis_types import MeshingMap

from qdesignoptimizer.utils.utils import get_save_path

# Closing Ansys hfss
from qdesignoptimizer.utils.utils import close_ansys

close_ansys()

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)

# map for finer meshing
meshing_map = [
    MeshingMap(component_class=CoupledLineTee, mesh_names=CoupledLineTee_mesh_names)
]

design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=get_save_path("out/", n.CHIP_NAME),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS,
    meshing_map=meshing_map,
)

## Optimization step

In [ ]:
# number of runs of optimization and number of passes for simulation at each run
nbr_iterations = 15
group_passes = 8
delta_f = 0.001
for i in range(nbr_iterations):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

## Results

### Eigenmodes

In [ ]:
design_analysis.get_eigenmode_results()

### Cross Kerr

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

## Update parameters

In [ ]:
design_analysis.overwrite_parameters()

## Close

In [ ]:
close_ansys()